# Solutions for Unit 3 Services Part 2

<img src="../img/robotignite_logo_text.png"/>

## Index: 
* <a href="#SolutionExercise3-2">Solution Exercise 3.2</a>

<p id="SolutionExercise3-2"></p>

## Solution Exercise 3.2

* The objective of this exercise 3.2 is to create a service that when called, BB8 robot moves in a square like trajectory.

Nothing to comment.

* You can work on a new package or use the one you created for exercise 3.1, called **unit_3_services**.

In this case, we are going to work on the package **unit_3_services**. ALthough note that you could work on a new package, or even separate the **service server** into one package, and the **service client** into another package. As you wish. We work on the same package for be clear in the explanation.

* Create a python file that has a class inside that allows the movement of the BB8 in a square  <a href="#fig-3.1">{Fig-3.1}</a>. This class could be called, for reference, **MoveBB8**. And the python code that contains it, could called <a href="#move_bb8_py">move_bb8.py</a>.<br>
To move BB-8, you just have to write in the **/cmd_vel** Topic, as you did in Unit1 Topics.<br>
Bear in mind that although this is a simulation, BB-8 has weight and, therefore, it won't stop immediately due to inertia.<br>
Also, when turning, friction and inertia will be playing a role. Bear in mind that by only moving through /cmd_vel, you don't have a way of checking if it turned the way you wanted (it's called an open loop system).
Unless, of course, you find a way to have some positional feedback information. That's a challenge for advanced AstroMech builders (if you want to try, think about using the /odom topic).<br>
But for considering the movement Ok, you just have to more or less move in a square, doesnt have to be perfect.

So first of all we create the python file **move_bb8.py** in the package **unit_3_services**. You can put it inside the **scripts** folder or inside the **src** folder. This depends on whether you intend to use this class as a python module that will be accessed outside this package by other packages or not.<br>
If you want it to be used outside this **unit_3_services**, it's advisibale to put it inside the src folder, inside a folder called like the package, with a **__init__.py** file. Because this is outside the scope of this unit, we will stick to the basic **scripts** location. You will learn how to do the more complex method in the **sphero project**.

Here you have the python file with the class **MoveBB8**, that moves the BB8 in a square in open loop.

<p style="background:#3B8F10;color:white;" id="move_bb8_py">**Python Program: move_bb8.py** </p><br>

In [ ]:
#!/usr/bin/env python
import rospy
from geometry_msgs.msg import Twist
import time

class MoveBB8():
    
    def __init__(self):
        self.bb8_vel_publisher = rospy.Publisher('/cmd_vel', Twist, queue_size=1)
        self.ctrl_c = False
        rospy.on_shutdown(self.shutdownhook)
        self.rate = rospy.Rate(10) # 10hz
    
    
    
    def publish_once_in_cmd_vel(self, cmd):
        """
        This is because publishing in topics sometimes fails teh first time you publish.
        In continuos publishing systems there is no big deal but in systems that publish only
        once it IS very important.
        """
        while not self.ctrl_c:
            connections = self.bb8_vel_publisher.get_num_connections()
            if connections > 0:
                self.bb8_vel_publisher.publish(cmd)
                rospy.loginfo("Cmd Published")
                break
            else:
                self.rate.sleep()
    
    
    def shutdownhook(self):
            # works better than the rospy.is_shut_down()
            self.stop_bb8()
            self.ctrl_c = True

    def stop_bb8(self):
        rospy.loginfo("shutdown time! Stop the robot")
        cmd = Twist()
        cmd.linear.x = 0.0
        cmd.angular.z = 0.0
        self.publish_once_in_cmd_vel(cmd)


    def move_x_time(self, moving_time, linear_speed=0.2, angular_speed=0.2):
        
        cmd = Twist()
        cmd.linear.x = linear_speed
        cmd.angular.z = angular_speed
        
        rospy.loginfo("Moving Forwards")
        self.publish_once_in_cmd_vel(cmd)
        time.sleep(moving_time)
        self.stop_bb8()
        rospy.loginfo("######## Finished Moving Forwards")
    
    def move_square(self):
        
        i = 0
        while not self.ctrl_c and i < 4:
            # Move Forwards
            self.move_x_time(moving_time=2.0, linear_speed=0.2, angular_speed=0.0)
            # Stop
            self.move_x_time(moving_time=4.0, linear_speed=0.0, angular_speed=0.0)
            # Turn 
            self.move_x_time(moving_time=3.5, linear_speed=0.0, angular_speed=0.2)
            # Stop
            self.move_x_time(moving_time=0.1, linear_speed=0.0, angular_speed=0.0)
            
            i += 1
        rospy.loginfo("######## Finished Moving in a Square")
        

            
if __name__ == '__main__':
    rospy.init_node('move_bb8_test', anonymous=True)
    movebb8_object = MoveBB8()
    try:
        movebb8_object.move_square()
    except rospy.ROSInterruptException:
        pass


<p style="background:#3B8F10;color:white;" id="move_bb8_py">END **Python Program: move_bb8.py** </p><br>

* Add a service server that accepts an <b>Empty</b> Service messages and activates the square movement. This service could be called **/move_bb8_in_square**<br>
This activation will be done through a call to the Class **MoveBB8**.<br>
For that, you have to create a very similar python file as <a href="#prg-3-7">simple_service_server.py</a>. You could call it **bb8_move_in_square_service_server.py**.

So the first step is to create a python file called <a href="#bb8_move_in_square_service_server_py">bb8_move_in_square_service_server.py</a>, which will be a modified version of the example given in **example 3.7**.

<p style="background:#3B8F10;color:white;" id="bb8_move_in_square_service_server_py">**Python Program: bb8_move_in_square_service_server.py** </p>

In [ ]:
#! /usr/bin/env python

import rospy
from std_srvs.srv import Empty, EmptyResponse # you import the service message python classes generated from Empty.srv.
from move_bb8 import MoveBB8

def my_callback(request):
    rospy.loginfo("The Service move_bb8_in_square has been called")
    movebb8_object = MoveBB8()
    movebb8_object.move_square()
    rospy.loginfo("Finished service move_bb8_in_square that was called called")
    return EmptyResponse() # the service Response class, in this case EmptyResponse
    #return MyServiceResponse(len(request.words.split())) 

rospy.init_node('service_move_bb8_in_square_server') 
my_service = rospy.Service('/move_bb8_in_square', Empty , my_callback) # create the Service called move_bb8_in_square with the defined callback
rospy.loginfo("Service /move_bb8_in_square Ready")
rospy.spin() # mantain the service open.

<p style="background:#3B8F10;color:white;">END **Python Program: bb8_move_in_square_service_server.py** </p>

Once you have it , test it through a **call** in the WebConsole to the service **/move_bb8_in_square**.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>
<br>

In [ ]:
rosservice call /move_bb8_in_square [TAB]+[TAB]

You should get BB8 moving like so:

<img src="../img/basic_unit3_exercise3-2.gif"/>

* Create a launch file called **start_bb8_move_in_square_service_server.launch**. Inside it you have to start a node that launches the **bb8_move_in_square_service_server.py**. 

So this is the <a href="#start_bb8_move_in_square_service_server_launch">start_bb8_move_in_square_service_server.launch</a> that launches the **bb8_move_in_square_service_server.py**.

<p style="background:#3B8F10;color:white;" id="start_bb8_move_in_square_service_server_launch">**Launch Program: start_bb8_move_in_square_service_server.launch** </p>

In [ ]:
<launch>
    <!-- Start Service Server for move_bb8_in_square service -->
    <node pkg="unit_3_services" type="bb8_move_in_square_service_server.py" name="service_move_bb8_in_square_server"  output="screen">
    </node>
</launch>

<p style="background:#3B8F10;color:white;">END **Launch Program: start_bb8_move_in_square_service_server.launch** </p>

Test it by launching this launch **start_bb8_move_in_square_service_server.launch** and calling the service as before. Is the exact same procedure , only that we are launching the service server through a launch file, instead of directly through a python file.

* Create a new python code, called **bb8_move_in_square_service_client.py**, that calls the service **/move_bb8_in_square**. Remember how it was done in **Unit3 Services Part1**.<br>
Then generate a new launch file, called **call_bb8_move_in_square_service_server.launch**, that executes the **bb8_move_in_square_service_client.py** through a node.

The <a href="#bb8_move_in_square_service_client_py">bb8_move_in_square_service_client.py</a> should be similar to this:

<p style="background:#3B8F10;color:white;" id="bb8_move_in_square_service_client_py">**Python Program: bb8_move_in_square_service_client.py** </p>

In [ ]:
#! /usr/bin/env python
import rospkg
import rospy
from std_srvs.srv import Empty, EmptyRequest # you import the service message python classes generated from Empty.srv.

rospy.init_node('service_move_bb8_in_square_client') # Initialise a ROS node with the name service_client
rospy.wait_for_service('/move_bb8_in_square') # Wait for the service client /move_bb8_in_square to be running
move_bb8_in_square_service_client = rospy.ServiceProxy('/move_bb8_in_square', Empty) # Create the connection to the service
move_bb8_in_square_request_object = EmptyRequest() # Create an object of type EmptyRequest

result = move_bb8_in_square_service_client(move_bb8_in_square_request_object) # Send through the connection the path to the trajectory file to be executed
print result # Print the result given by the service called

<p style="background:#3B8F10;color:white;">END **Python Program: bb8_move_in_square_service_client.py** </p>

The <a href="#call_bb8_move_in_square_service_server_launch">call_bb8_move_in_square_service_server.launch</a> should be like this:

<p style="background:#3B8F10;color:white;" id="call_bb8_move_in_square_service_server_launch">**Launch Program: call_bb8_move_in_square_service_server.launch** </p>

In [ ]:
<launch>
    <!-- Start Service Server for move_bb8_in_square service -->
    <node pkg="unit_3_services" type="bb8_move_in_square_service_client.py" name="service_move_bb8_in_square_client"  output="screen">
    </node>
</launch>

<p style="background:#3B8F10;color:white;" id="start_bb8_move_in_square_service_server_launch">**Launch Program: call_bb8_move_in_square_service_server.launch** </p>

* When launched **call_bb8_move_in_square_service_server.launch**, bb8 should move in a square.

This should works exactly thesame way as the other calls you have performed. But in this case you launch **start_bb8_move_in_square_service_server.launch** and the in another terminal you launch **call_bb8_move_in_square_service_server.launch**.

<p id="SolutionExercise3-3"></p>